# 유니콘 기업을 찾아보자

## 가설

##### 유니콘 기업은 '효율적인' 기업일 것이다
* 적은 직원으로 높은 매출액을 내는 기업이 유니콘일 가능성이 크다

##### 유니콘 기업은 '인적자원'을 중시하는 기업일 것이다
* 1인당 연금보험료를 높게 측정하고 있는 기업이 유니콘일 가능성이 크다

## '1인당_매출액' 과 '1인당_연금보험료'를 활용한 점수계산

##### Case1. 두 변수 각각의 ranking을 합산
* 1인당 매출액 순위 + 1인당 연금보험료 순위 = 점수

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('company_nps_data.csv',encoding='cp949')

In [3]:
data['1인당 매출액'] = data['연매출액'] / data['월별_직원수'] 
data['1인당 연금보험료'] = data['월별_연금보험료'] / data['월별_직원수']
data[np.isinf(data)] = 0 #0으로 나눠서 발생하는 무한값 제거

In [4]:
data = pd.read_csv('company_nps_data.csv',encoding='cp949')
data['1인당 매출액'] = data['연매출액'] / data['월별_직원수'] 
data['1인당 연금보험료'] = data['월별_연금보험료'] / data['월별_직원수']
data[np.isinf(data)] = 0 #0으로 나눠서 발생하는 무한값 제거


In [5]:
def binding(data):
     #회사ID를 기준으로 묶기
    dct = {}
    for i in data.groupby('회사ID'):
        dct[i[0]] = i[1].sort_values(['년도', '월']) #데이터를 시간순서대로 활용할려면 dct에 저장되는 데이터를 index순서대로 활용하면 됨
    #회사ID별 평균값 도출
    dct_mean = {}
    for i in dct.keys():
        dct_mean[i] = dct[i].mean()
    dct_mean_df = pd.DataFrame(dct_mean).T #dict를 dataframe으로 전환   
    return dct_mean_df

In [6]:
dct_mean_df = binding(data)

In [7]:
dct_mean_df['1인당 매출액 순위'] = ss.rankdata(dct_mean_df['1인당 매출액'])
dct_mean_df['1인당 연금보험료 순위'] = ss.rankdata(dct_mean_df['1인당 연금보험료'])
dct_mean_df['점수'] = dct_mean_df['1인당 매출액 순위'] + dct_mean_df['1인당 연금보험료 순위']

In [8]:
dct_mean_df.sort_values(by=['점수'],ascending = False).head(10)

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수,1인당 매출액,1인당 연금보험료,1인당 매출액 순위,1인당 연금보험료 순위,점수
63,63.0,1.132878e+10,2018.000000,6.500000,4.101282e+08,1007.166667,1.124874e+07,407239.467496,4988.0,4995.0,9983.0
60154,60154.0,5.080587e+07,2018.000000,8.000000,2.477320e+06,6.333333,8.171949e+06,390976.190476,4977.0,4991.0,9968.0
294649,294649.0,3.836737e+09,2016.894737,6.763158,1.029398e+08,270.473684,1.418880e+07,380563.578053,4990.0,4976.0,9966.0
293402,293402.0,3.413610e+07,2018.000000,6.500000,1.501150e+06,4.000000,9.102960e+06,381050.000000,4983.0,4978.0,9961.0
5452,5452.0,2.523765e+08,2018.000000,7.500000,1.534591e+07,39.700000,6.358736e+06,386545.948530,4972.0,4987.0,9959.0
64324,64324.0,5.540396e+07,2018.000000,8.500000,1.872410e+06,5.000000,1.108079e+07,374482.000000,4985.0,4963.0,9948.0
40860,40860.0,1.709144e+08,2018.000000,6.500000,2.570220e+06,7.000000,2.441635e+07,367174.285714,4997.0,4944.0,9941.0
381180,381180.0,1.459657e+09,2016.894737,6.763158,6.377902e+07,170.605263,8.575456e+06,374044.555746,4979.0,4962.0,9941.0
126831,126831.0,2.019423e+09,2016.894737,6.763158,2.276844e+08,584.078947,3.457156e+06,389804.704049,4916.0,4990.0,9906.0
419998,419998.0,3.538393e+09,2017.000000,6.500000,3.026178e+08,812.444444,4.355804e+06,372419.518004,4943.0,4959.0,9902.0


##### Case2. 두 변수를 0-1사이의 값으로 스케일링 한 뒤 단순합산
* scaled_1인당 매출액 + scaled_1인당 연금보험료 = 점수

In [9]:
min_max_scaler = MinMaxScaler()
fitted = min_max_scaler.fit(data[['1인당 매출액','1인당 연금보험료']])
data[['1인당 매출액','1인당 연금보험료']] = min_max_scaler.transform(data[['1인당 매출액','1인당 연금보험료']])
data['점수'] = data['1인당 매출액'] + data['1인당 연금보험료']

In [10]:
dct_mean_df2 = binding(data)

In [11]:
dct_mean_df2.sort_values(by=['점수'],ascending = False).head(10)

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수,1인당 매출액,1인당 연금보험료,점수
40860,40860.0,1.709144e+08,2018.000000,6.500000,2.570220e+06,7.000000,0.496413,0.751726,1.248140
146654,146654.0,6.397783e+08,2016.894737,6.763158,8.863051e+06,26.552632,0.491542,0.679348,1.170890
63,63.0,1.132878e+10,2018.000000,6.500000,4.101282e+08,1007.166667,0.244937,0.840030,1.084967
294649,294649.0,3.836737e+09,2016.894737,6.763158,1.029398e+08,270.473684,0.301086,0.781236,1.082323
567650,567650.0,1.136639e+08,2018.000000,8.000000,5.035667e+05,3.000000,0.753696,0.312429,1.066124
297874,297874.0,1.162102e+09,2016.894737,6.763158,8.311988e+06,37.500000,0.621420,0.428617,1.050037
433492,433492.0,9.753366e+07,2016.384615,6.884615,1.021949e+06,4.076923,0.542416,0.481355,1.023771
64324,64324.0,5.540396e+07,2018.000000,8.500000,1.872410e+06,5.000000,0.241729,0.767833,1.009562
60154,60154.0,5.080587e+07,2018.000000,8.000000,2.477320e+06,6.333333,0.186176,0.804186,0.990362
293402,293402.0,3.413610e+07,2018.000000,6.500000,1.501150e+06,4.000000,0.203956,0.782309,0.986265


##### 두 케이스에 공통으로 상위권을 차지한 5개 기업 추출

In [16]:
a1 = dct_mean_df.sort_values(by=['점수'],ascending = False).head(9).index
a2 = dct_mean_df2.sort_values(by=['점수'],ascending = False).head(9).index
unicon_company = list(set(a1).intersection(a2))
print(str(unicon_company)+"이 유니콘 기업 입니다")

[64324, 294649, 60154, 40860, 63]이 유니콘 기업 입니다
